In [1]:
import json
import os
import requests
from urllib.parse import urlparse

In [2]:
# Directory to save downloaded MP3 files
output_directory = 'downloaded_mp3s'

# Create the output directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

In [3]:
with open("har_data.json", "r", encoding="utf-8") as file:
    input_data = json.load(file)

input_data

mp3_urls = []

entries = input_data["log"]["entries"]

for i, entry in enumerate(entries):
    request = entry["request"]
    url = request["url"]
    if url.endswith(".mp3"):
        mp3_urls.append(url)

mp3_urls

['http://learn101.org/sound/swahili/swahili_shirt.mp3',
 'http://learn101.org/sound/swahili/swahili_sweater.mp3',
 'http://learn101.org/sound/swahili/swahili_jacket.mp3',
 'http://learn101.org/sound/swahili/swahili_coat.mp3',
 'http://learn101.org/sound/swahili/swahili_socks.mp3',
 'http://learn101.org/sound/swahili/swahili_shoes.mp3',
 'http://learn101.org/sound/swahili/swahili_trousers.mp3',
 'http://learn101.org/sound/swahili/swahili_pajamas.mp3',
 'http://learn101.org/sound/swahili/swahili_belt.mp3',
 'http://learn101.org/sound/swahili/swahili_underwear.mp3',
 'http://learn101.org/sound/swahili/swahili_hat.mp3',
 'http://learn101.org/sound/swahili/swahili_skirt.mp3',
 'http://learn101.org/sound/swahili/swahili_Arabic.mp3',
 'http://learn101.org/sound/swahili/swahili_Morocco.mp3',
 'http://learn101.org/sound/swahili/swahili_Chinese.mp3',
 'http://learn101.org/sound/swahili/swahili_China.mp3',
 'http://learn101.org/sound/swahili/swahili_English.mp3',
 'http://learn101.org/sound/swahi

In [4]:
# Custom headers to be added to the requests
custom_headers = {
    'Accept': '*/*',
    'Accept-Encoding': 'identity;q=1, *;q=0',
    'Accept-Language': 'en-US,en;q=0.9',
    'Dnt': '1',
    'Host': 'learn101.org',
    'Proxy-Connection': 'keep-alive',
    'Range': 'bytes=0-',
    'Referer': 'http://learn101.org/sound/swahili/swahili_one.mp3',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
}

# Download MP3 files from the URLs
# mp3_url = mp3_urls[0]
for mp3_url in mp3_urls:
    try:
        custom_headers["Referer"] = mp3_url
        response = requests.get(mp3_url, headers=custom_headers, stream=True)
        if response.status_code == 200 or response.status_code == 206:
            # Extract the filename from the URL
            url_path = urlparse(mp3_url).path
            file_name = os.path.join(output_directory, os.path.basename(url_path))

            # Save the MP3 file locally with the same name as in the URL
            with open(file_name, 'wb') as mp3_file:
                for chunk in response.iter_content(chunk_size=1024):
                    if chunk:
                        mp3_file.write(chunk)
            print(f'Downloaded: {file_name}')
        else:
            print(f'Failed to download: {mp3_url}, Status Code: {response.status_code}')
    except Exception as e:
        print(f'Error downloading: {mp3_url}, Error: {str(e)}')

print('Download complete.')


Downloaded: downloaded_mp3s\swahili_shirt.mp3
Downloaded: downloaded_mp3s\swahili_sweater.mp3
Downloaded: downloaded_mp3s\swahili_jacket.mp3
Downloaded: downloaded_mp3s\swahili_coat.mp3
Downloaded: downloaded_mp3s\swahili_socks.mp3
Downloaded: downloaded_mp3s\swahili_shoes.mp3
Downloaded: downloaded_mp3s\swahili_trousers.mp3
Downloaded: downloaded_mp3s\swahili_pajamas.mp3
Downloaded: downloaded_mp3s\swahili_belt.mp3
Downloaded: downloaded_mp3s\swahili_underwear.mp3
Downloaded: downloaded_mp3s\swahili_hat.mp3
Downloaded: downloaded_mp3s\swahili_skirt.mp3
Downloaded: downloaded_mp3s\swahili_Arabic.mp3
Downloaded: downloaded_mp3s\swahili_Morocco.mp3
Downloaded: downloaded_mp3s\swahili_Chinese.mp3
Downloaded: downloaded_mp3s\swahili_China.mp3
Downloaded: downloaded_mp3s\swahili_English.mp3
Downloaded: downloaded_mp3s\swahili_England.mp3
Downloaded: downloaded_mp3s\swahili_French.mp3
Downloaded: downloaded_mp3s\swahili_France.mp3
Downloaded: downloaded_mp3s\swahili_German.mp3
Downloaded: do